<a href="https://colab.research.google.com/github/shaobaili3/342Ass1/blob/master/78A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieve Dataset

## Link Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
v_path = "/content/drive/My Drive/NLPa2/v.csv"

## Parse dataset

In [4]:
train_path = "/content/drive/My Drive/NLPa2/train.csv"
test_path = "/content/drive/My Drive/NLPa2/test.csv"
val_path = "/content/drive/My Drive/NLPa2/val.csv" #3000 in total
result_path = "/content/drive/My Drive/NLPa2/result.csv"

def read_data(file_name, n_sample):
    f = open(file_name)
    documents = f.readlines()
    documents.pop(0) # Remove title, i.e., sentence, NER
    input_data = []

    target_data = []

    for i in documents[0: n_sample]:
      temp_words = []
      temp_labels = []
      line = i.replace('\n','').rsplit(',', 1)
      input = line[0]
      target = line[1]
      
      for word in input.split():
        temp_words.append(word)
      for label in target.split():
        temp_labels.append(label)
      
      input_data.append(temp_words)
      target_data.append(temp_labels)

    return input_data, target_data

def get_test_strings():
  test_data = []
  test_string = []
  f = open(test_path)
  documents = f.readlines()
  documents.pop(0) # Remove title, i.e., sentence, NER
  for i in documents:
    words = i.replace('\n','')[:-1]
    test_string.append(words)
    temp = []
    for word in words.split():
    	temp.append(word)
    test_data.append(temp)
  return test_data, test_string

train_data, target_y_train = read_data(train_path, 3000) # 3000 in total
validation_data, target_y_validation = read_data(val_path, 700)
#test_data, target_y_test = read_data(val_path,700)
test_data , test_string = get_test_strings()

print("train_data", len(train_data))
print("validation_data", len(validation_data))
print("test_data", len(test_data))

# print(train_data[-1])
# print(target_y_train[0:5])
# print(validation_data[0:5])
# print(target_y_validation[0:5])
# print(test_data[0:5])


train_data 3000
validation_data 700
test_data 3684


## Generate index

In [0]:
# Get word_to_ix and tag_to_ix
START_TAG = "<START>"
STOP_TAG = "<STOP>"

def get_index(all_data, all_target):
  word_to_ix = {}
  for sentence in all_data:
      for word in sentence:
          word = word.lower()
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
  word_list = list(word_to_ix.keys())
  tag_to_ix = {START_TAG:0, STOP_TAG:1}
  for tags in all_target:
      for tag in tags:
          if tag not in tag_to_ix:
              tag_to_ix[tag] = len(tag_to_ix)
  return word_to_ix, tag_to_ix, word_list

# Get index from data
def to_index(data, to_ix):
  input_index_list = []
  for sent in data:
    input_index_list.append([to_ix[w] for w in sent])
  return input_index_list

# Embeddings


In [6]:
import gensim.downloader as api
import numpy as np

word_emb_model = api.load("glove-twitter-200") 

def get_embedding_matrix(EMBEDDING_DIM):
  embedding_matrix = []
  for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
  embedding_matrix = np.array(embedding_matrix)
  embedding_matrix.shape
  return embedding_matrix

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# BI-LSTM baseline model

## Model

In [0]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, embedding_matrix):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

def initialize_model(EMBEDDING_DIM, HIDDEN_DIM, embedding_matrix):
  model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, embedding_matrix).to(device)
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
  return model, optimizer

## Train

# Validation functions

In [0]:
import numpy as np

def cal_acc(model, input_index, output_index):
  predicts = []
  totalNum = 0 
  correctNum = 0
  for i, idxs in enumerate(input_index):
    sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
    predict = model(sentence_in)[1]
    predicts.append(predict)
    output = output_index[i]
    for i, v in enumerate(predict):
      totalNum += 1
      if v == output[i]:
        correctNum += 1
  first = []
  second = []
  for a in output_index:
    for o in a:
      first.append(o)
  for a in predicts:
    for o in a:
      second.append(o)
  return first, second, correctNum/totalNum


def get_result(model, input_index):
  predicts = []
  for i, idxs in enumerate(input_index):
    sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
    predict = model(sentence_in)[1]
    predicts.append(predict)
  return predicts

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

# Kaggle

In [0]:
def decode_result(results):
  test_results = []
  for result in results:
    test_result = decode_output(result)
    test_results.append(test_result)
  return test_results

def write_csv(result):
  f = open(result_path, "w")
  f.write("Id,Predicted\n")
  index = 0
  for r in result:
    for c in r:
      f.write(str(index) + "," + c + "\n")
      index += 1
  f.close()
  print("Result Updated in Drive!")

# Processing

In [10]:
# Data retrieval
train_data, target_y_train = read_data(train_path, 3000) # 3000 in total
validation_data, target_y_validation = read_data(val_path, 700)
#test_data, target_y_test = read_data(val_path,700)
test_data , test_string = get_test_strings()

all_data = train_data + validation_data + test_data
all_target = target_y_train + target_y_validation
word_to_ix, tag_to_ix, word_list = get_index(all_data, all_target)

train_input_index =  to_index(train_data,word_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)
#test_output_index = to_index(target_y_test,tag_to_ix)

EMBEDDING_DIM = 200
HIDDEN_DIM = 200
embedding_matrix = get_embedding_matrix(EMBEDDING_DIM)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [0]:
# Initialize model
model, optimizer = initialize_model(EMBEDDING_DIM, HIDDEN_DIM, embedding_matrix)

In [0]:
import datetime

def train(model, epoch_size):
  bestScore = 0.93
  for epoch in range(epoch_size):  #20
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    #_, _, val_acc = cal_acc(model,val_input_index,val_output_index)
    _, _, val_acc = cal_acc(model,v_input_index,v_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()
    if val_acc > bestScore:
      bestScore = val_acc
      print("best!!!: " , bestScore)
      torch.save(model, "/content/drive/My Drive/NLPa2/model.pt")
    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

In [0]:
from sklearn.metrics import classification_report

def get_kaggle():
  test_input_index = to_index(test_data, word_to_ix)
  results = get_result(model, test_input_index)
  write_csv(decode_result(results))

def validate():
  y_true,y_pred,_ = cal_acc(model, val_input_index, val_output_index)
  y_true_decode = decode_output(y_true)
  y_pred_decode = decode_output(y_pred)
  print(classification_report(y_true_decode,y_pred_decode,digits=4))

In [0]:
# train(model, 100)
# validate()

In [15]:
v_data, target_y_v = read_data(v_path, 36854)
v_input_index = to_index(v_data,word_to_ix)
v_output_index = to_index(target_y_v,tag_to_ix)

def validate2():
  y_true,y_pred,_ = cal_acc(model, v_input_index, v_output_index)
  y_true_decode = decode_output(y_true)
  y_pred_decode = decode_output(y_pred)
  print(classification_report(y_true_decode,y_pred_decode,digits=4))

validate2()

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       I-LOC     0.0351    0.4390    0.0650      1925
      I-MISC     0.0000    0.0000    0.0000       918
       I-ORG     0.2043    0.0188    0.0345      2496
       I-PER     0.0000    0.0000    0.0000      2773
           O     0.8074    0.4685    0.5929     38554

    accuracy                         0.4062     46666
   macro avg     0.2094    0.1853    0.1385     46666
weighted avg     0.6794    0.4062    0.4944     46666



In [16]:
# train(model, 200) 
model = torch.load("/content/drive/My Drive/NLPa2/model.pt")
model.eval()

BiLSTM_CRF(
  (word_embeds): Embedding(14882, 200)
  (lstm): LSTM(200, 200, bidirectional=True)
  (hidden2tag): Linear(in_features=400, out_features=7, bias=True)
)

SUBMISSION

In [17]:
get_kaggle()
!pip install -q kaggle
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/NLPa2/kaggle.json ~/.kaggle/
!kaggle competitions submit -c 2020-comp5046-a2 -f /content/drive/My\ Drive/NLPa2/result.csv -m "test"

Result Updated in Drive!
mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 384k/384k [00:01<00:00, 337kB/s]
Successfully submitted to COMP5046-2020S1-A2